In [1]:
%%HTML
<style>
    div#notebook-container    { width: 80%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>

In [2]:
%load_ext autoreload

## 量子状態と期待値の測定

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# 必要なライブラリ その1: qulacs
# 適宜pip installしてください
from qulacs import QuantumState, QuantumCircuit
from qulacs import Observable
from qulacs.observable import create_observable_from_openfermion_text

# 必要なライブラリ その2: openfermion
from openfermion import QubitOperator

In [4]:
n_qubit = 3 # number of qubits
nsamp_per_axis = 1 # sample per measurement axis
Nsamp_tot = 10000 # total number of measurements

# prepare random quantum state
psi = QuantumState(n_qubit)
psi.set_Haar_random_state(seed = 123)

In [5]:
# 量子状態  = 2^n_qubit 次元の複素ベクトル
print(psi)

# numpy 形式でexportもできる
print(psi.get_vector())

 *** Quantum State ***
 * Qubit Count : 3
 * Dimension   : 8
 * State vector : 
   (0.057055,0.315604)
 (-0.0520797,0.389076)
(0.0417348,-0.0369918)
 (-0.0452564,0.453675)
 (0.120369,-0.0283307)
  (-0.50341,-0.411403)
 (-0.149604,-0.103599)
(-0.230856,-0.0877067)

[ 0.057055  +0.31560426j -0.0520797 +0.38907646j  0.0417348 -0.03699179j
 -0.04525635+0.45367519j  0.12036944-0.02833065j -0.50341032-0.41140299j
 -0.14960351-0.10359856j -0.23085551-0.08770668j]


In [6]:
# 演算子を定義する。
operator = QubitOperator("X0 Z1 Y2")
op_ql = create_observable_from_openfermion_text(str(operator))

# 量子状態 psiにおける 期待値 <psi | op | psi>
op_ql.get_expectation_value(psi)

0.05937732497116038

In [7]:
# 複数の演算子を定義
operators = [QubitOperator("X0 Z1 Y2"), QubitOperator("Z0 Y1 Y2"), QubitOperator("Z0 Y2")]
ops_ql = [create_observable_from_openfermion_text(str(_op)) for _op in operators]

# 複数の演算子に関する期待値計算
[_op_ql.get_expectation_value(psi) for _op_ql in ops_ql]

[0.05937732497116038, 0.3781249240449189, -0.7509136629735933]

実際の量子デバイスにおいても、以下の順序に従って期待値が測定されていく。
1. 量子状態を準備する
2. 測定したい演算子に対応して「測定軸」を決める
3. 測定軸に対応するユニタリゲートをかけて射影測定

一方で、測定したい演算子が複数ある場合、可換な演算子を同時に測定することができる。
例えば、上の例のように
\begin{eqnarray}
O_1 &=& X_0 Z_1 Y_2,\\
O_2 &=& Z_0 Y_1 Y_2, \\
O_3 &=& Z_0 Y_2
\end{eqnarray}
のように定義したとき、$O_2$と$O_3$の違いは1番目の$Y_1$の有無だけだから、同時測定が可能である。
実機の負担を軽減するには、同時測定が可能であるような演算子を、しっかり同時に測ってやることが必要となる。
そのようなプロトコルを与えるのが、Classical shadowを用いた測定スキームである。

## Classical shadow tomography

Classical shadowの最大の利点は、「与えられた物理量に対する測定」を行うのではなく、「ランダム化された測定結果を下に、任意の物理量の期待値が可能となる」点にある。
具体的な手順を見ていこう。

In [8]:
class LocalPauliShadowSampler_core(object):
    def __init__(
        self,
        n_qubit:int,
        state:QuantumState,
        Nshadow_tot: int,
        nshot_per_axis = 1,
    ):
        self.n_qubit = n_qubit
        self.state = state
        self.Ntot = Nshadow_tot
        self.m = nshot_per_axis
        
    def set_state(self, state):
        self.state = state

    def set_random_seed(self, seed):
        np.random.seed(seed)

    def generate_random_measurement_axis(self):
        """
        Creates a list of random measurement axis.
        For 3-qubit system, this looks like, e.g.,
          [1, 3, 2], 
        which tells you to measure in 
          [X, Z, Y]
        basis.

        return:
            List[n_qubit]
        """
        return list(np.random.randint(1, 4, size = self.n_qubit))  
        
    def _sample_digits(self, meas_axis, nshot_per_axis = 1):
        """
            returns the measurement result at meas_axis.
            attributes:
                meas_axis: List of axes
                nshot_per_axis: number of measurement per axis
        """
        meas_state = QuantumState(self.n_qubit)
        meas_state.load(self.state)

        meas_circuit = QuantumCircuit(self.n_qubit)
        
        # それぞれのqubitに測定用のuntiaryを作用
        for qindex in range(self.n_qubit):
            _axis = meas_axis[qindex]
            if _axis == 1:
                # X軸測定のためのユニタリ
                meas_circuit.add_H_gate(qindex)
            elif _axis == 2:
                # Y軸測定のためのユニタリ
                meas_circuit.add_Sdag_gate(qindex)
                meas_circuit.add_H_gate(qindex)

        meas_circuit.update_quantum_state(meas_state)
        
        # サンプリングに対応するシミュレーション
        digits = meas_state.sampling(count = nshot_per_axis)
        return digits

In [9]:
# This is my ShadowSampler implementation
#from shadow_generator import LocalPauliShadowSampler

Nshadow_tot = 10000 # 合計のランダム測定回数
nshot_per_axis = 1 # 軸ごとの測定回数 (この値は通常1にとる)
shadow_sampler = LocalPauliShadowSampler_core(n_qubit, psi, Nshadow_tot, nshot_per_axis)

shadow_sampler.set_random_seed(seed = 1234)

In [10]:
# 完全にランダムに、それぞれのqubitにおける測定軸を決定する
# 例えば[3,2,1] → [Z, Y, X]軸で測定せよ、という意味になる
meas_axes = [shadow_sampler.generate_random_measurement_axis() for _ in range(Nshadow_tot)]
print("meas_axes[:5] = ", meas_axes[:5])

meas_axes[:5] =  [[3, 2, 1], [1, 1, 2], [2, 2, 3], [3, 3, 1], [1, 3, 3]]


In [11]:
# それぞれの測定軸で測定を行い、その結果をbitarrayとして保存しておく
sample_digits = [shadow_sampler._sample_digits(_meas_ax, nshot_per_axis = nshot_per_axis) for _meas_ax in meas_axes]
sample_digits = sum(sample_digits, []) # 整形
bitstring_array = [format(_samp,"b").zfill(shadow_sampler.n_qubit) for _samp in sample_digits]
samples = np.array([[int(_b) for _b in _bitstring][::-1] for _bitstring in bitstring_array])
print("samples[:5] = ", samples[:5])

samples[:5] =  [[1 0 1]
 [0 1 1]
 [1 0 0]
 [1 1 1]
 [0 0 1]]


この時点では、具体的な物理量の期待値は何ももとまっていないことに注意しよう。
ランダムな測定軸と、測定結果に対応するランダムなビット列があるだけである。

これらを適切に組み合わせる (post-processing)と、物理量の期待値を推定することができる。

In [12]:
from typing import List

def _estimate_single_pauli_product(operator: QubitOperator, meas_axes: List[int], samples: List[int], K=None):
    """
    attributes:
        operator: QubitOperator
        meas_axes: list of measurement axes
        
    """
    global n_qubit, Nshadow_tot
    paulistring = as_paulistring(operator)

    assert np.array(meas_axes).shape == np.array(samples).shape
    pauli_ids = create_pauli_id_from_openfermion(operator, n_qubit)
    pauli = np.array([pauli_ids]*Nshadow_tot)

    # This is the core of estimator, which corresponds to Algotihm 1 of
    # https://arxiv.org/abs/2006.15788
    arr = (np.array(meas_axes) ==np.array(pauli)) * 3
    arr = (-1) ** np.array(samples) * arr
    arr += (np.array(pauli) == 0)
    val_array =  np.prod(arr, axis = -1).copy()


    if K is not None:
        exp = median_of_means(val_array, K)
    else:
        exp = np.mean(val_array)        

    return exp

def as_paulistring(operator):
    return " ".join([action + str(site) for site, action in list(operator.terms.keys())[0]])

def if_single_pauli_product(operator):
    return len(str(operator).split("+")) == 1        

def create_pauli_id_from_openfermion(operator, n_qubit):
    assert if_single_pauli_product(operator), "input is not single product of pauli."

    terms = list(operator.terms.keys())[0]
    qidx_list = []
    pauli_id_list = []
    plist = ["I", "X", "Y", "Z"]
    for _term in terms:
        qidx_list.append(_term[0])
        pauli_id_list.append(plist.index(_term[1]))    
    
    ret = [0]*n_qubit
    for _qid, _pid in zip(qidx_list, pauli_id_list):
        ret[_qid] = _pid
    return ret    

In [13]:
# ランダムに決めた測定軸および測定結果を用いて、任意の物理量に関する推定ができる。
# ここでは便宜上、Single Paulistringのみを受け取る関数を作る（一般の演算子を測定するには、単純に線形結合をとれば良い）。
op1 = QubitOperator("Z0 Y1 X2") 
op2 = QubitOperator("Z0 Y1")
op3 = QubitOperator("X0 Y2")

op1_est = _estimate_single_pauli_product(op1, meas_axes, samples)
op2_est = _estimate_single_pauli_product(op2, meas_axes, samples)
op3_est = _estimate_single_pauli_product(op3, meas_axes, samples)
print(op1_est)
print(op2_est)
print(op3_est)

0.3699
0.0396
0.252


In [14]:
# 厳密な期待値と比べてみる。 Nshadow_sampに関して、その誤差は 1/sqrt{Nshadow_samp}でスケールする。
op1_ql = create_observable_from_openfermion_text(str(op1))
op2_ql = create_observable_from_openfermion_text(str(op2  + 1e-6 * QubitOperator("Z2")))# qulacsの都合で、qubit数よりも添字が小さい演算子でget_expectation_valueするときはこれが必要)
op3_ql = create_observable_from_openfermion_text(str(op3))

op1_exp = op1_ql.get_expectation_value(psi)
op2_exp = op2_ql.get_expectation_value(psi)
op3_exp = op3_ql.get_expectation_value(psi)

# いずれも誤差は 1/sqrt{Nshadow_samp}のオーダー
print(f"shadow : {op1_est:.5f}, exact : {op1_exp:.5f}, error = {np.abs(op1_exp - op1_est):5f} ")
print(f"shadow : {op2_est:.5f}, exact : {op2_exp:.5f}, error = {np.abs(op2_exp - op2_est):5f} ")
print(f"shadow : {op3_est:.5f}, exact : {op3_exp:.5f}, error = {np.abs(op3_exp - op3_est):5f} ")

shadow : 0.36990, exact : 0.38130, error = 0.011399 
shadow : 0.03960, exact : 0.04970, error = 0.010101 
shadow : 0.25200, exact : 0.30082, error = 0.048816 


## 課題
ここまでは、完全にランダムに決定された測定軸に対するClassical shadowを想定していた。
一方で、測定したい物理量にある程度のあたりがついている場合には、測定軸に偏りを持たせた方が効率が良いという旨が [arXiv:2006.15788](https://arxiv.org/abs/2006.15788)で指摘されている。具体的には以下の擬コードに対応する（記号などは論文を読んでください）。
ちなみに、上で実装されているのは Algorithm1に対応している。
<img src="./Algorithm2_LBCS.png">

このAlgorithm2を実装したい。そこで、以下の手続きに従って `Sampler`と `Estimator`の両方を設計した上で、正しい物理量の計算を実行できることを確認せよ。
1. $i$番目のqubitに対して、確率を[ 1/3, 1/3, 1/3]で測定軸を決めるのではなく、$\beta_i = [\beta_{i, X}, \beta_{i, Y}, \beta_{i, Z}]$に従って出力せよ。\\要するに、 `LocallyBiasedPauliShadowSampler_core.generate_random_measurement_axis`を正しく設計せよ。
2. $\beta_i$を不均等にする影響を `_estimate_single_pauli_product_biased`に反映させ、正しい期待値を計算せよ。要するに、Algorithm2における $f(P, Q, \beta)$の関数系を反映させたestimatorを作成せよ。

In [ ]:
# Shadow sampler

class LocallyBiasedPauliShadowSampler_core(object):
    def __init__(
        self,
        n_qubit:int,
        state:QuantumState,
        Nshadow_tot: int,
        nshot_per_axis = 1,
    ):
        self.n_qubit = n_qubit
        self.state = state
        self.Ntot = Nshadow_tot
        self.m = nshot_per_axis
        
    def set_state(self, state):
        self.state = state

    def set_random_seed(self, seed):
        np.random.seed(seed)

    def generate_random_measurement_axis(self, beta):
        """
        attributes:
            beta : [n_qubit, 3], array of probability distibution

        return:
            List[n_qubit]
        """
        
        ############################
        # Generate measurement axes according to probability distribution beta.
        # beta[i] is a 3-dimensiontal array corresponding to probability distribution of X, Y, Z axis
        ############################
        
        return meas_axes
        
    def _sample_digits(self, meas_axis, nshot_per_axis = 1):
        """
            returns the measurement result at meas_axis.
            attributes:
                meas_axis: List of axes
                nshot_per_axis: number of measurement per axis
        """
        meas_state = QuantumState(self.n_qubit)
        meas_state.load(self.state)

        meas_circuit = QuantumCircuit(self.n_qubit)
        for qindex in range(self.n_qubit):
            _axis = meas_axis[qindex]
            if _axis == 1:
                meas_circuit.add_H_gate(qindex)
            elif _axis == 2:
                meas_circuit.add_Sdag_gate(qindex)
                meas_circuit.add_H_gate(qindex)

        meas_circuit.update_quantum_state(meas_state)
        digits = meas_state.sampling(count = nshot_per_axis)
        return digits

In [ ]:
# Estimator

def _estimate_single_pauli_product_biased(operator: QubitOperator, meas_axes: List[int], samples: List[int], beta):
    """
    attributes:
        operator: QubitOperator
        meas_axes: list of measurement axes
        samples: list of measurement results
    """
    
    ############################
    # estimate expectation value
    ############################
    
    return exp